# Step 3: Weather Data Fetching (ERA5)

In this step, we fetch daily **weather data** from the Open-Meteo ERA5 API for each grid cell.  
The parameters collected are:
- Maximum & minimum temperature
- Precipitation (rainfall)
- Maximum windspeed
- Average relative humidity

The weather dataset (`weather.csv`) will be used with fire events to train machine learning models.


In [ ]:
import pandas as pd
import requests
import time
from tqdm import tqdm

grid = pd.read_csv("grid.csv")

# ===== USER SETTINGS =====
START_DATE = "2024-11-01"
END_DATE   = "2025-01-13"
OUT_FILE   = "weather.csv"
# =========================

daily_vars = [
    "temperature_2m_max",
    "temperature_2m_min",
    "precipitation_sum",
    "windspeed_10m_max",
    "relative_humidity_2m_mean"
]
daily = ",".join(daily_vars)

records = []

for _, row in tqdm(grid.iterrows(), total=len(grid)):
    lat, lon, cid = row["centroid_lat"], row["centroid_lon"], row["cell_id"]

    url = (
        "https://archive-api.open-meteo.com/v1/era5"
        f"?latitude={lat}&longitude={lon}"
        f"&start_date={START_DATE}&end_date={END_DATE}"
        f"&daily={daily}&timezone=UTC"
    )

    try:
        r = requests.get(url, timeout=60)
        if r.status_code != 200:
            print(f"Skipped {cid} (HTTP {r.status_code})")
            continue
        js = r.json()
        if "daily" not in js:
            print(f"No daily data for {cid}")
            continue
        df = pd.DataFrame(js["daily"])
        df.insert(0, "cell_id", cid)
        records.append(df)
    except Exception as e:
        print(f"Error at {cid}: {e}")
        continue

    time.sleep(0.5)  # slow down requests slightly

# Save if we got something
if records:
    weather = pd.concat(records, ignore_index=True)
    weather.to_csv(OUT_FILE, index=False)
    print(f"Saved weather data to {OUT_FILE} with shape {weather.shape}")
    print(weather.head())
else:
    print("No weather data fetched")


 86%|████████▌ | 904/1050 [18:00<45:49, 18.83s/it]

Error at c904: HTTPSConnectionPool(host='archive-api.open-meteo.com', port=443): Read timed out. (read timeout=60)


100%|██████████| 1050/1050 [20:44<00:00,  1.19s/it]


Saved weather data to weather.csv with shape (77626, 7)
  cell_id        time  temperature_2m_max  temperature_2m_min  \
0      c1  2024-11-01                31.4                17.4   
1      c1  2024-11-02                30.9                16.1   
2      c1  2024-11-03                30.2                16.8   
3      c1  2024-11-04                29.7                16.6   
4      c1  2024-11-05                28.9                18.7   

   precipitation_sum  windspeed_10m_max  relative_humidity_2m_mean  
0                0.0               14.2                         56  
1                0.0                9.3                         62  
2                0.0                7.5                         69  
3                0.0                7.5                         74  
4                0.0                8.8                         77  
